This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [2]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 49.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 kB 27.8 MB/s eta 0:00:00


In [1]:
import os
import numpy as np
import pandas as pd

os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [15]:
from langchain.schema import SystemMessage, HumanMessage
# Instantiate the ChatOpenAI model
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=1.2,
    max_tokens=2048
)

# Define system and human messages using the proper classes
system_message = SystemMessage(
    content=(
        "Please generate a unique real estate listing with different neighborhood names, prices, and house descriptions. "
        "Avoid using the same phrases and try different writing styles. "
        "Ensure that no two listings are too similar."
    )
)

human_message = HumanMessage(
    content=(
        "Please generate a real estate listing similar to the following format:\n\n"
        "Neighborhood: Green Oaks\n"
        "Price: $800,000\n"
        "Bedrooms: 3\n"
        "Bathrooms: 2\n"
        "House Size: 2,000 sqft\n\n"
        "Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. "
        "This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. "
        "Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. "
        "The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. "
        "Embrace sustainable living without compromising on style in this Green Oaks gem.\n\n"
        "Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, "
        "community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. "
        "With easy access to public transportation and bike lanes, commuting is a breeze."
        
        "Neighborhood: Willow Creek\n"
        "Price: $1,200,000\n"
        "Bedrooms: 5\n"
        "Bathrooms: 4\n"
        "House Size: 3,000 sqft"
        "Description: Experience grandeur in this spectacular 5-bedroom, 4-bathroom residence located in the sought-after Willow Creek neighborhood. "
        "Embrace the exquisite craftsmanship and attention to detail throughout this stately home, featuring a grand foyer, formal dining room, and a spacious family room with a cozy fireplace."
        "The gourmet chef's kitchen is a culinary oasis with high-end appliances and an oversized island for gatherings. The backyard retreat offers a sparkling pool, spa, and manicured gardens, providing the perfect setting for relaxing and entertaining."
        "Live lavishly in this Willow Creek treasure.\n"
        "Neighborhood Description: Willow Creek is known for its upscale estates, tree-lined streets, and proximity to top-rated schools and upscale shopping centers."
        "Spend weekends exploring the nearby Willow Creek Country Club or meander through the boutique shops at The Pier Shopping Plaza. Enjoy the convenience of fine dining and entertainment options just a stone's throw away, creating an ideal balance of luxury and livability in Willow Creek.\n"
    )
)

df = pd.DataFrame(columns=["Neighborhood", "Price", "Bedrooms", "Bathrooms", "House Size", "Description", "Neighborhood Description"])

for _ in range(10):
    response = llm.invoke([system_message, human_message])
    generated_text = response.content
    
    estate_data = {}
    for line in generated_text.split("\n"):
        if ":" in line:
            key, value = line.split(":", 1)
            estate_data[key.strip()] = value.strip()
            
    df = pd.concat([df, pd.DataFrame([estate_data])], ignore_index=True)
    
print(df)

          Neighborhood       Price Bedrooms Bathrooms  House Size  \
0      Oakwood Estates  $1,500,000        6       4.5  4,200 sqft   
1  Cedar Grove Heights    $950,000        4       3.5  2,800 sqft   
2    Harborview Bluffs  $1,450,000        6         5  3,800 sqft   
3      Pinecrest Grove    $700,000        3       2.5  2,200 sqft   
4    Harbor View Hills    $950,000        4         3  2,500 sqft   
5   Riverfront Estates  $1,500,000        6         5  4,000 sqft   
6          Cedar Ridge    $950,000        4         3  2,500 sqft   
7      Pinecrest Woods    $950,000        4         3  2,500 sqft   
8    Riverbend Estates  $1,450,000        6         5  4,500 sqft   
9  Harbor View Estates  $1,500,000        6       5.5  4,200 sqft   

                                         Description  \
0  Discover unparalleled sophistication in this e...   
1  Step into luxury and comfort in this exquisite...   
2  Indulge in luxury living in the prestigious Ha...   
3  Embrace comfo

In [16]:
df.to_csv("generated_listings.csv", index=False)

In [27]:
# Continue previous work
df = pd.read_csv("generated_listings.csv")

In [18]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document

embedding_function = OpenAIEmbeddings()

chroma_db = Chroma(
    collection_name="real_estate_listings",
    embedding_function=embedding_function,
    persist_directory="./chroma_db"
)

In [19]:
def store_listings_in_chroma(df):
    documents = []
    
    for _, row in df.iterrows():
        listing_text = f"""
        Neighborhood: {row['Neighborhood']}
        Price: {row['Price']}
        Bedrooms: {row['Bedrooms']}
        Bathrooms: {row['Bathrooms']}
        House Size: {row['House Size']}
        Description: {row['Description']}
        Neighborhood Description: {row['Neighborhood Description']}
        """
        doc = Document(page_content=listing_text, metadata=row.to_dict())  
        documents.append(doc)

    chroma_db.add_documents(documents)
    print("✅ All estate listing has been saved in ChromaDB！")

store_listings_in_chroma(df)

✅ All estate listing has been saved in ChromaDB！


In [20]:
questions = [   
                "How big do you want your house to be?" 
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
preset_answers = [
                "A comfortable four-bedroom house with a spacious kitchen and a cozy living room.",
                "A quiet neighborhood, good local schools, and convenient shopping options.",
                "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
                "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
                "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
            ]

In [21]:
user_query = " ".join(preset_answers)

k = 1

matching_listings = chroma_db.similarity_search(user_query, k)

print("\n🏠 Recommended estate listings:")
for i, result in enumerate(matching_listings):
    print(f"\n Estates {i+1}:")
    print(result.page_content)
    print(f" Metadata: {result.metadata}")


🏠 Recommended estate listings:

 Estates 1:

        Neighborhood: Maple Grove
        Price: $950,000
        Bedrooms: 4
        Bathrooms: 3
        House Size: 2,500 sqft
        Description: Step into this meticulously renovated 4-bedroom, 3-bathroom home in the desirable Maple Grove neighborhood. This residence exudes modern elegance with a touch of rustic charm. The open-concept living area features vaulted ceilings, large windows, and a cozy fireplace. The gourmet kitchen is a chef's dream, complete with stainless steel appliances and quartz countertops. Retreat to the master suite with a spa-like ensuite bathroom and a private balcony overlooking the lush backyard oasis. Entertain guests in the landscaped outdoor space with a fire pit and built-in BBQ area. Embrace luxury living in Maple Grove.
        Neighborhood Description: Maple Grove is a serene neighborhood known for its tree-lined streets, friendly atmosphere, and easy access to parks and walking trails. Enjoy a morni

Step 6: Personalizing Listing Descriptions

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature = 1,
    max_tokens = 256
)

top_listing = matching_listings[0]
metadata = top_listing.metadata

system_message = SystemMessage(
    content = "You are a real estate expert. Based on a given property listing and a buyer's preferences, rewrite the listing description to better highlight features important to the buyer. Do not alter factual information."
)

human_message = HumanMessage(
    content=f"Buyer Preferences: {user_query}\n\n"
            f"Original Listing:\n"
            f"Neighborhood: {metadata['Neighborhood']}\n"
            f"Price: {metadata['Price']}\n"
            f"Bedrooms: {metadata['Bedrooms']}\n"
            f"Bathrooms: {metadata['Bathrooms']}\n"
            f"House Size: {metadata['House Size']}\n"
            f"Description: {metadata['Description']}\n"
            f"Neighborhood Description: {metadata['Neighborhood Description']}\n\n"
            f"Please rewrite the description to emphasize aspects that align with the buyer's preferences."
)

personalized_description = llm([system_message, human_message]).content

print("\n🏡 **Personalized Real Estate Listing:**")
print(f" **Neighborhood:** {metadata['Neighborhood']}")
print(f" **Price:** {metadata['Price']}")
print(f" **Bedrooms:** {metadata['Bedrooms']}, 🛁 **Bathrooms:** {metadata['Bathrooms']}")
print(f" **Size:** {metadata['House Size']} sqft")
print(f" **Customized Description:** {personalized_description}")


🏡 **Personalized Real Estate Listing:**
 **Neighborhood:** Maple Grove
 **Price:** $950,000
 **Bedrooms:** 4, 🛁 **Bathrooms:** 3
 **Size:** 2,500 sqft sqft
 **Customized Description:** Located in the peaceful Maple Grove neighborhood, this inviting 4-bedroom, 3-bathroom home is priced at $950,000 and offers 2,500 sqft of living space. The residence boasts a spacious kitchen perfect for cooking enthusiasts, featuring stainless steel appliances and quartz countertops. The cozy living room with vaulted ceilings and a fireplace provides a welcoming retreat. The house includes a backyard ready for gardening, a two-car garage, and a modern, energy-efficient heating system for comfortable living. Situated in a quiet area with good local schools and convenient shopping options, this home also provides easy access to a reliable bus line, a nearby major highway, and bike-friendly roads. Enjoy a balance of suburban tranquility and urban amenities like restaurants and theaters in this ideal locat

Recheck the buyer Preferences, 
"A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
"A quiet neighborhood, good local schools, and convenient shopping options.",
"A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
"Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
"A balance between suburban tranquility and access to urban amenities like restaurants and theaters."

We can find it works.

Step into luxury in this sprawling 6-bedroom, 5-bathroom estate situated in the prestigious Riverbend Estates neighborhood. This magnificent residence exudes sophistication and grandeur, boasting elegant finishes, high ceilings, and expansive living areas. The grand foyer welcomes you into a home designed for both comfort and style, featuring a gourmet kitchen with top-of-the-line appliances, a formal dining room for hosting lavish soirées, and a cozy family room with a stone fireplace. Escape to the lavish master suite retreat with a spa-like bathroom and private balcony overlooking the manicured grounds. Outdoor entertainment abounds with a covered patio, sparkling pool, and lush landscaping, creating a resort-like atmosphere perfect for gatherings and relaxation.",


Riverbend Estates is known for its upscale properties, riverside views, and exclusivity. Enjoy a prestigious lifestyle with access to the nearby Riverbend Golf Club, upscale dining options, and luxury boutiques. Take a leisurely stroll along the riverfront or spend weekends hosting elegant events in your own private estate. Experience the epitome of luxury living in Riverbend Estates."

In [41]:
different_answers = [
                "I’m looking for a spacious home with six bedrooms and five bathrooms to accommodate my family and guests comfortably, offering plenty of room for both private retreats and entertaining."
                "The three most important factors for me are luxury finishes, ample living space, and a prime location in an exclusive neighborhood like Riverbend Estates."
                "I would love to have a gourmet kitchen, a stone fireplace in the family room, a spa-like master suite, and a backyard with a pool and covered patio for outdoor entertainment."
                "I value easy access to major roads, as well as proximity to public transportation options like bus lines or bike-friendly routes, making commuting convenient."
                "I prefer a tranquil, suburban setting that still offers easy access to urban amenities, such as restaurants, theaters, and shopping centers, for a perfect balance of peace and convenience."
            ]

In [42]:
user_query = " ".join(different_answers)

k = 1

matching_listings = chroma_db.similarity_search(user_query, k)

print("\n🏠 Recommended estate listings:")
for i, result in enumerate(matching_listings):
    print(f"\n Estates {i+1}:")
    print(result.page_content)
    print(f" Metadata: {result.metadata}")


🏠 Recommended estate listings:

 Estates 1:

        Neighborhood: Riverbend Estates
        Price: $1,450,000
        Bedrooms: 6
        Bathrooms: 5.0
        House Size: 4,500 sqft
        Description: Step into luxury in this sprawling 6-bedroom, 5-bathroom estate situated in the prestigious Riverbend Estates neighborhood. This magnificent residence exudes sophistication and grandeur, boasting elegant finishes, high ceilings, and expansive living areas. The grand foyer welcomes you into a home designed for both comfort and style, featuring a gourmet kitchen with top-of-the-line appliances, a formal dining room for hosting lavish soirées, and a cozy family room with a stone fireplace. Escape to the lavish master suite retreat with a spa-like bathroom and private balcony overlooking the manicured grounds. Outdoor entertainment abounds with a covered patio, sparkling pool, and lush landscaping, creating a resort-like atmosphere perfect for gatherings and relaxation.
        Neighbor

In [43]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature = 1,
    max_tokens = 256
)

top_listing = matching_listings[0]
metadata = top_listing.metadata

system_message = SystemMessage(
    content = "You are a real estate expert. Based on a given property listing and a buyer's preferences, rewrite the listing description to better highlight features important to the buyer. Do not alter factual information."
)

human_message = HumanMessage(
    content=f"Buyer Preferences: {user_query}\n\n"
            f"Original Listing:\n"
            f"Neighborhood: {metadata['Neighborhood']}\n"
            f"Price: {metadata['Price']}\n"
            f"Bedrooms: {metadata['Bedrooms']}\n"
            f"Bathrooms: {metadata['Bathrooms']}\n"
            f"House Size: {metadata['House Size']}\n"
            f"Description: {metadata['Description']}\n"
            f"Neighborhood Description: {metadata['Neighborhood Description']}\n\n"
            f"Please rewrite the description to emphasize aspects that align with the buyer's preferences."
)

personalized_description = llm([system_message, human_message]).content

print("\n🏡 **Personalized Real Estate Listing:**")
print(f" **Neighborhood:** {metadata['Neighborhood']}")
print(f" **Price:** {metadata['Price']}")
print(f" **Bedrooms:** {metadata['Bedrooms']}, 🛁 **Bathrooms:** {metadata['Bathrooms']}")
print(f" **Size:** {metadata['House Size']} sqft")
print(f" **Customized Description:** {personalized_description}")


🏡 **Personalized Real Estate Listing:**
 **Neighborhood:** Riverbend Estates
 **Price:** $1,450,000
 **Bedrooms:** 6, 🛁 **Bathrooms:** 5.0
 **Size:** 4,500 sqft sqft
 **Customized Description:** Immerse yourself in opulence within this expansive 6-bedroom, 5-bathroom haven nestled in the sought-after Riverbend Estates enclave, priced at $1,450,000. This lavish abode captures the essence of luxury living with upscale finishes and sprawling living spaces, offering an impressive 4,500 sqft of living area. The luxurious gourmet kitchen is a culinary masterpiece, complemented by a stunning stone fireplace in the family room for cozy gatherings.

Indulge in the tranquility of the sumptuous master suite featuring a spa-like bathroom and a private balcony for serene moments. Outdoor entertainment awaits in the backyard oasis, complete with a sparkling pool and a covered patio, ideal for hosting guests or unwinding in style. 

Located in the prestigious Riverbend Estates neighborhood, this pro